# Figure 4: analysis of IBCM in turbulent backgrounds
This version is with a six-odor, turbulent background. 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl
import os, colorsys, json
from os.path import join as pj

from mpl_toolkits.axes_grid1.inset_locator import inset_axes

In [ ]:
# Resources
root_dir = pj("..", "..")
data_folder = pj(root_dir, "results", "for_plots")
panels_folder = "panels/"
params_folder = pj(root_dir, "results", "common_params")

do_save_plots = True

# Aesthetic parameters

In [ ]:
# rcParams
with open(pj(params_folder, "olfaction_rcparams.json"), "r") as f:
    new_rcParams = json.load(f)
plt.rcParams.update(new_rcParams)

# color maps
with open(pj(params_folder, "back_colors.json"), "r") as f:
    all_back_colors = json.load(f)
back_color = all_back_colors["back_color"]
back_color_samples = all_back_colors["back_color_samples"]
back_palette = all_back_colors["back_palette"]

with open(pj(params_folder, "orn_colors.json"), "r") as f:
    orn_colors = json.load(f)
    
with open(pj(params_folder, "inhibitory_neuron_two_colors.json"), "r") as f:
    neuron_colors = np.asarray(json.load(f))
with open(pj(params_folder, "inhibitory_neuron_full_colors.json"), "r") as f:
    neuron_colors_full = np.asarray(json.load(f))

with open(pj(params_folder, "model_colors.json"), "r") as f:
    model_colors = json.load(f)
with open(pj(params_folder, "model_nice_names.json"), "r") as f:
    model_nice_names = json.load(f)
model_colors["random"] = "k"
model_nice_names["random"] = "Rand. odors"

In [ ]:
n_neu = np.load(pj(data_folder, 
                    "sample_turbulent_ibcm_simulation.npz"))["cbars_gamma"].shape[1]
n_components, n_orn = np.load(pj(data_folder, 
                    "sample_turbulent_ibcm_simulation.npz"))["back_vecs"].shape

In [ ]:
# Extra aesthetic parameters for this figure

# More legend rcParams: make everything smaller by 30 %
plt.rcParams["patch.linewidth"] = 0.75
legend_rc = {"labelspacing":0.5, "handlelength":2.0, "handleheight":0.7, 
             "handletextpad":0.8, "borderaxespad":0.5, "columnspacing":2.0}
for k in legend_rc:
    plt.rcParams["legend."+k] = 0.75 * legend_rc[k]

new_color = "r"
linestyles = ["-", "--", ":", (0, (5, 1, 2, 1)), "-."]
neuron_styles = linestyles + [(0, (1, 2, 1, 2))]

## Smoothing function

In [ ]:
def moving_var(points, kernelsize, ddof=1, boundary="free"):
    """ Computing the variance of time series points in a sliding window.

    Args:
        points (np.ndarray): the data points
        kernelsize (int): odd integer giving the window size. 
        boundary (str): how to deal with points within kernelsize//2 of edges
            "shrink": the window for a point within distance d < w
                is shrunk symmetrically to a kernel of size d
            "free": the window is asymmetric, full on the inside and clipped
                on the side near the edge.
            "noflux": these points are set to the value of the closest point
                with full window (i.e. distance kernelsize//2 of the edge)

    Returns:
        var_points (np.ndarray): standard deviation at every point
    """
    var_points = np.zeros(points.shape)
    # To compute std, we need to compute the average too
    avg_points = np.zeros(points.shape)
    if kernelsize < 3: raise ValueError("Need larger kernel for variance")
    if kernelsize % 2 == 0:  # if an even number was given
        kernelsize -= 1
    w = kernelsize // 2  # width
    end = avg_points.shape[0]  # index of the last element

    if boundary not in ["shrink", "free", "noflux"]:
        raise ValueError("Unknown boundary {}".format(boundary))

    # Smooth the middle points using slicing.
    # First store second moment in var_points
    var_points[w:end - w] = points[w:end - w]**2
    avg_points[w:end - w] = points[w: end - w]
    for j in range(w):  # Add points around the middle one
        avg_points[w:-w] += points[w - j - 1:end - w - j - 1]
        avg_points[w:-w] += points[w + j + 1:end - w + j + 1]
        var_points[w:-w] += points[w - j - 1:end - w - j - 1]**2
        var_points[w:-w] += points[w + j + 1:end - w + j + 1]**2

        # Use the loop to treat the two points at a distance j from boundaries
        if j < w and j > 0 and boundary == "shrink":
            avg_points[j] = np.sum(points[0:2*j + 1], axis=0) / (2*j + 1)
            var_points[j] = (np.sum(points[0:2*j + 1]**2, axis=0)
                    - avg_points[j]**2 * (2*j + 1)) / (2*j + 1 - ddof)
            avg_points[-j - 1] = np.sum(points[-2*j - 1:], axis=0) / (2*j + 1)
            var_points[-j - 1] = (np.sum(points[-2*j - 1:]**2, axis=0)
                    - avg_points[-j - 1]**2 * (2*j + 1)) / (2*j + 1 - ddof)
        elif j < w and boundary == "free":
            avg_points[j] = np.sum(points[0:j + w + 1], axis=0) / (j + w + 1)
            var_points[j] = (np.sum(points[0:j + w + 1]**2, axis=0)
                    - avg_points[j]**2 * (j + w + 1)) / (j + w + 1 - ddof)
            avg_points[-j - 1] = np.sum(points[-j - w - 1:], axis=0) / (j + w + 1)
            var_points[-j - 1] = (np.sum(points[-j - w - 1:]**2, axis=0)
                    - avg_points[-j - 1]**2 * (j + w + 1)) / (j + w + 1 - ddof)

    # Normalize the middle points by kernelsize - ddof
    avg_points[w:end - w] /= kernelsize
    var_points[w:end - w] /= (kernelsize - ddof)

    # Set the edge points to the nearest full point if boundary is no flux
    if boundary == "noflux":
        var_points[:w] = var_points[w]
        var_points[-w:] = var_points[-w]

    # Then subtract the average squared, taking ddof into account once
    var_points[w:end - w] -= (avg_points[w:end - w]**2
                                * kernelsize / (kernelsize - ddof))

    return var_points

# Panel A: IBCM learning $c_{\gamma}$s time series


In [ ]:
# Load example concentration time series
ex = np.load(pj(data_folder, "sample_turbulent_ibcm_simulation.npz"))
tser_example = np.arange(*ex["tser_range"])
cgammaser_example = ex["cbars_gamma"]
n_i_ibcm = cgammaser_example.shape[1]
yser_example = ex["yser"]
analytical_cs_cn = ex["cs_cn"]
specif_gammas = ex["specif_gammas"]
back_components = ex["back_vecs"]
n_b = back_components.shape[0]

# Time units per simulation step, for plotting, in ms
dt_u = 10.0  # ms

In [ ]:
# Show three neurons
fig = plt.figure()
gs = fig.add_gridspec(3, 4)
fig.set_size_inches(plt.rcParams["figure.figsize"][0]*1.25, 
                    plt.rcParams["figure.figsize"][1])
ax = fig.add_subplot(gs[:, :3])
axi = fig.add_subplot(gs[1, 3:])

#ax.axhline(0.0, ls="-", color=(0.8,)*3, lw=0.8)
t_axis = tser_example*dt_u/1000/60
legend_styles = [[0,]*6, [0,]*6, [0,]*6]
i_highlights = [2, 16, 21]  # Neurons to highlight
neuron_colors3 = neuron_colors_full[[8, 17, 23]]
clr_back = back_palette[-1]
plot_skp = 25

# plot all other neurons first, skip some points
for i in range(n_i_ibcm):
    if i in i_highlights: 
        continue
    else: 
        for j in range(n_b):
            ax.plot(t_axis[::plot_skp], cgammaser_example[::plot_skp, i, j], color=clr_back, 
                ls="-", alpha=1.0-0.1*j, lw=plt.rcParams["lines.linewidth"]-j*0.1)

# Now plot the highlighted neuron
for j in range(n_b):
    for i in range(len(i_highlights)):
        li, = ax.plot(t_axis[::plot_skp], cgammaser_example[::plot_skp, i_highlights[i], j], 
                      color=neuron_colors3[i], ls="-", alpha=1.0-0.1*j, 
                      lw=plt.rcParams["lines.linewidth"]-j*0.1)
        legend_styles[i][j] = li

# Annotate with analytical results
#for j in range(2):
#    ax.axhline(analytical_cs_cn[j], lw=1.0, ls="-.", color="k")
#ax.annotate(r"Analytical $h_\mathrm{ns}$", xy=(t_axis[-5], analytical_cs_cn[1]-0.4), 
#            ha="right", va="top", color="k", size=6)
#ax.annotate(r"Analytical $h_\mathrm{s}$", xy=(t_axis[-5], analytical_cs_cn[0]+0.4), 
#            ha="right", va="bottom", color="k", size=6)
ax.set(xlabel="Time (min)", 
       ylabel=r"Alignments $\bar{h}_{i\gamma} = \mathbf{\bar{m}}_i \cdot \mathbf{s}_{\gamma}$")

axi.tick_params(labelleft=True, labelbottom=False, labeltop=True)
for i in range(len(i_highlights)):
    for j in range(n_b):
        li = legend_styles[i][j]
        axi.plot([0.0+j, 0.5+j], [0.8*i, 0.8*i], color=li.get_color(), 
                 ls=li.get_linestyle(), alpha=li.get_alpha(), lw=li.get_linewidth())
for side in ["bottom", "left", "top", "right"]:
    axi.spines[side].set_visible(False)
axi.tick_params(axis="both", length=0, pad=3)
axi.set_xticks(np.arange(0.25, n_b+0.25, 2.0))
#axi.set_xticklabels([r"$\mathbf{\bar{m}}_i \cdot \mathbf{s}_a$", r"$\mathbf{\bar{m}} \cdot \mathbf{s}_b$"])
axi.set_xticklabels(list(range(0, n_b, 2)))
axi.set_xlabel(r"Odor $\gamma$", size=6, labelpad=3)
axi.xaxis.set_label_position('top') 
axi.set_yticks([0, 0.8, 1.6])
axi.invert_yaxis()
axi.set_yticklabels(["${}$".format(i) for i in i_highlights])
#axi.set_yticklabels(["Neuron 2", "Neuron 1"])
axi.set_ylabel("Neuron $i$", size=6, labelpad=1)

gs.tight_layout(fig, w_pad=-0.2)
if do_save_plots:
    fig.savefig(pj(panels_folder, "sample_turbulent_cgamma_series.pdf"), 
           transparent=True, bbox_inches="tight")
plt.show()
plt.close()

# Panel B: dot products summary
To really show the specificity. Could go to supplementary, or stay if we move analysis of individual simulations to a supplementary figure. 

In [ ]:
transient = int(3*tser_example.size/4)
cgammas_matrix = np.mean(cgammaser_example[transient:], axis=0)

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(plt.rcParams["figure.figsize"][0]*0.6, plt.rcParams["figure.figsize"][1])
# Extent: left, right, bottom, top
# Greyscale version
#ax.imshow(cgammas_matrix, cmap="Greys", aspect=0.6, extent=(0.5, n_b+0.5, 0.5, n_i_ibcm))
#ax.set_xticks(list(range(1, n_b+1)))
# Colorful version: add patches manually with fill_between. 
# Color highlighted neurons, leave others grayscale!
normed_matrix = (cgammas_matrix - cgammas_matrix.min()) / (cgammas_matrix.max() - cgammas_matrix.min())
for i in range(n_i_ibcm):
    # Full rainbow version
    #cmap = sns.light_palette(neuron_colors_full[i], as_cmap=True)
    # Version where only highlights are colored
    if i in i_highlights:
        cmap = sns.light_palette(neuron_colors3[i_highlights.index(i)], as_cmap=True)
    else:
        cmap = sns.color_palette("Greys", as_cmap=True)
    for j in range(n_b):
        ax.fill_between([-0.5+j, 0.5+j], -0.5+i, 0.5+i, color=cmap(normed_matrix[i, j]))
        
ax.set_xlim([-0.6, -0.6+n_b])
ax.set_ylim([-0.6, -0.6+n_i_ibcm])
ax.set_xticks(list(range(0, n_b)))
ax.set_yticks(list(range(0, 19, 2)) + [21, 23])
for i, lbl in enumerate(ax.get_yticklabels()):
    if int(lbl.get_text()) in i_highlights:
        clr = neuron_colors3[i_highlights.index(int(lbl.get_text()))]
        lbl.set_color(clr)
        ax.yaxis.get_ticklines()[i].set_color(clr)
ax.set(xlabel=r"Component $\gamma$", ylabel="IBCM neuron index $i$")
cbar = fig.colorbar(mpl.cm.ScalarMappable(
    norm=mpl.colors.Normalize(cgammas_matrix.min(), cgammas_matrix.max()), 
    cmap="Greys"), ax=ax, label=r"Alignments ${\bar{h}}_{i\gamma}$ after 45 min", aspect=30, pad=0.1)
fig.tight_layout()
if do_save_plots:
    fig.savefig(pj(panels_folder, "sample_turbulent_cgamma_matrix.pdf"), 
            transparent=True, bbox_inches="tight", bbox_extra_artists=(cbar.ax,))
plt.show()
plt.close()

# Panels C-D: PCA model analysis
Show that model is doing its job, when we're lucky. 

In [ ]:
ex2 = np.load(pj(data_folder, "sample_turbulent_biopca_simulation.npz"))
true_pca_values = ex2["true_pca_values"]
true_pca_vectors = ex2["true_pca_vectors"]
learnt_pca_values = ex2["learnt_pca_values"]
learnt_pca_vectors = ex2["learnt_pca_vectors"]
align_error_ser = ex2["align_error_ser"]

In [ ]:
# First plot: eigenvalues
n_comp = learnt_pca_values.shape[1]
pca_palette = sns.color_palette("colorblind", n_colors=n_comp)
fig, ax = plt.subplots()

for i in range(n_comp):
    li, = ax.plot(tser_example*dt_u/1000/60, learnt_pca_values[:, i], label="Value {}".format(i),
                  lw=plt.rcParams["lines.linewidth"] - 0.5*i/n_comp, zorder=10-i, color=pca_palette[i])
    if true_pca_values[i] / true_pca_values.max() > 1e-12:
        ax.axhline(true_pca_values[i], ls="--", color=pca_palette[i], 
                   lw=plt.rcParams["lines.linewidth"] - 0.5*i/n_comp, zorder=n_comp-i)
ax.set(ylabel="Principal values, diag$(L^{-1})$", yscale="log", xlabel="Time (min)")
handles = [mpl.lines.Line2D([0], [0], color="grey", ls="-", label=r"BioPCA ($L^{-1}$ diagonal)", 
                            lw=plt.rcParams["lines.linewidth"]), 
          mpl.lines.Line2D([0], [0], color="grey", ls="--", label="True PCA", 
                          lw=plt.rcParams["lines.linewidth"])]
leg = ax.legend(handles=handles, frameon=False, fontsize=5.0)
leg.set_zorder(30)
ax.set_ylim([ax.get_ylim()[0]*0.8, ax.get_ylim()[1]])

fig.tight_layout()
if do_save_plots:
    fig.savefig(pj(panels_folder, "biopca_eigenvalues_turbulent_example.pdf"), 
            transparent=True, bbox_inches="tight")
plt.show()
plt.close()

In [ ]:
fig, ax = plt.subplots()
ax.plot(tser_example*dt_u/1000/60, align_error_ser, color="k")
ax.set(yscale="log", ylabel="Subspace alignment error", xlabel="Time (min)")

fig.tight_layout()
if do_save_plots:
    fig.savefig(pj(panels_folder, "biopca_align_error_turbulent_example.pdf"), 
            transparent=True, bbox_inches="tight")
plt.show()
plt.close()

# Supplementary panel: IBCM eigenvalues
This will go to supplementary. 

In [ ]:
# Load eigenvalues for this example
with open(pj(data_folder, "ibcm_eigenvalues_keys_turbulent_example.json"), "r") as f:
    ibcm_specif_keys = json.load(f)

ibcm_eig_values=ex["ibcm_eig_values"]

In [ ]:
fig, ax = plt.subplots()
reals, imags = np.real(ibcm_eig_values), np.imag(ibcm_eig_values)
ibcm_eig_values_specif1 = np.asarray([len(s) == 1 for s in ibcm_specif_keys], dtype=bool)
highlights = ibcm_eig_values_specif1
ax.axvline(0.0, ls="--", color="k", lw=1.0)
ax.axhline(0.0, ls="--", color="k", lw=1.0)
scaleup = 1e3
ax.plot(reals[highlights]*scaleup, imags[highlights]*scaleup, marker="*", 
        mfc=model_colors["ibcm"], mec=model_colors["ibcm"], 
        ls="none", label="One odor", ms=5)
ax.plot(reals[~highlights]*scaleup, imags[~highlights]*scaleup, marker="o", mfc="k", mec="k", 
       ls="none", label="0 or 2+ odors", ms=3, alpha=0.7)
for side in ("top", "right"):
    ax.spines[side].set_visible(False)
ax.legend(title="Specificity of\nthe fixed point", loc="center right", title_fontsize=6)
ax.set(xlabel=r"$\mathrm{Re}(\lambda_{\mathrm{max}})$    ($\times 10^{-3}$)", 
      ylabel=r"$\mathrm{Im}(\lambda_{\mathrm{max}})$     ($\times 10^{-3}$)")
fig.tight_layout()
# Separate notebook now plots these eigenvalues
#fig.savefig(pj(panels_folder, "ibcm_jacobian_max_eigenvalues_turbulent.pdf"), 
#           transparent=True, bbox_inches="tight")
plt.show()
plt.close()